In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import linear_model
# logistic regression is part of sklearn - linear model

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
df = pd.read_csv(os.path.join('C:\Users\morgane1\Desktop\policefatalities.csv'))

In [3]:
df

,Name,Cause of Death,Date of Death,State,City,...,Month of Death,Year of Death,Was the deceased armed?,Did the deceased have priors?,Was the officer involved fired or suspended?
0,Hue Dang,Car Accident,11/3/2015,New Jersey,"Hackensack, New Jersey",...,March,2015,No,Unclear,Unclear
1,Bien Cam Tran,Car Accident,3/9/2014,California,"Oakland, California",...,14-Sep,2014,Unclear,Unclear,Unclear
2,Jacqueline Reynolds,Car Accident,8/5/2013,Illinois,"Chicago, Illinois",...,13-May,2013,No,No,No
3,Bernard Moore,Car Accident,6/3/2015,Georgia,"Atlanta, Georgia",...,March,2015,No,Unclear,Unclear
4,Jacqueline Nichols,Car Accident,4/7/2014,Michigan,"Flint, Michigan",...,14-Jul,2014,Unclear,Unclear,Unclear
...,...,...,...,...,...,...,...,...,...,...,...
2350,Timothy Shad Griffis,Taser Gun,25/08/2014,Florida,"Lake City, Florida",...,14-Aug,2014,No,Yes,Yes
2351,Billy Collins,Taser Gun,29/5/15,Kentucky,"Louisa, Kentucky",...,May,2015,No,Unclear,Unclear
2352,Michael Tyrone Gallagher,Taser Gun,11/5/2015,North Carolina,"Enfield, North Carolina",...,May,2015,No,Unclear,Unclear
2353,Randall Torrence,Taser Gun,27/5/15,Kansas,"Kansas City, Kansas",...,May,2015,No,Unclear,Unclear


In [4]:
df.shape

(2355, 15)

In [5]:
list(df.columns.values)

['Name',
 'Cause of Death',
 'Date of Death',
 'State',
 'City',
 'Gender',
 'Age',
 'Race',
 'Photo',
 'News Story',
 'Month of Death',
 'Year of Death',
 'Was the deceased armed?',
 'Did the deceased have priors?',
 'Was the officer involved fired or suspended?']

In [6]:
df = df.rename(columns = {'Cause of Death':'cause_death', 'Date of Death': 'date_death', 'Month of Death': 'month_death', 'Year of Death':'year_death','Was the deceased armed?':'armed', 'Did the deceased have priors?':'priors','Was the officer involved fired or suspended?':'fired'})

In [7]:
list(df.columns.values)

['Name',
 'cause_death',
 'date_death',
 'State',
 'City',
 'Gender',
 'Age',
 'Race',
 'Photo',
 'News Story',
 'month_death',
 'year_death',
 'armed',
 'priors',
 'fired']

In [8]:
df.drop(['Name', 'date_death','City','News Story','Photo','month_death','year_death'], axis=1, inplace=True)
df.head()

,cause_death,State,Gender,Age,Race,armed,priors,fired
0,Car Accident,New Jersey,Female,64.0,Asian,No,Unclear,Unclear
1,Car Accident,California,Male,58.0,Asian,Unclear,Unclear,Unclear
2,Car Accident,Illinois,Female,54.0,Black,No,No,No
3,Car Accident,Georgia,Male,62.0,Black,No,Unclear,Unclear
4,Car Accident,Michigan,Female,64.0,Black,Unclear,Unclear,Unclear


In [9]:
#bucket the ages by groups. - ranges 
df.Age.value_counts()

26.0     86
34.0     81
27.0     79
28.0     76
29.0     75
         ..
97.0      1
95.0      1
93.0      1
77.0      1
107.0     1
Name: Age, dtype: int64

In [10]:
print df.Age.mean()
print df.Age.median()
print df.Age.max()
print df.Age.min()

36.9122422115
34.0
107.0
5.0


In [11]:
print df[(df.Age == 107)]

    cause_death     State Gender    Age   Race armed   priors    fired
338    Shooting  Arkansas   Male  107.0  Black   Yes  Unclear  Unclear


In [12]:
df.State.value_counts()
#could just look at california and texas (or look at top 5)
#could also look at red / blue states 

California        398
Texas             237
Florida           187
Arizona           104
Illinois           68
                 ... 
New Hampshire       4
Virgin Islands      2
Rhode Island        2
Vermont             2
North Dakota        1
Name: State, dtype: int64

In [13]:
df.armed.value_counts()

Yes         1482
Unclear      550
No           275
Disputed      48
Name: armed, dtype: int64

In [14]:
df.priors.value_counts()

Unclear    1803
Yes         435
No          117
Name: priors, dtype: int64

In [15]:
df.cause_death.value_counts()
#consider all as fatalities and - can be dropped. Don't drop but don't remove. 

Shooting                  2076
Taser Gun                   97
Car Accident                69
Physical Confrontation      63
Other                       50
Name: cause_death, dtype: int64

In [16]:
# do I need to change car Accident to Car accident? same below for male?

In [17]:
df.Gender.value_counts()


Male       2210
Female      142
Unknown       3
Name: Gender, dtype: int64

In [18]:
df.Race.value_counts()

White               766
Unknown             713
Black               511
Latino              319
Asian                27
Native American      16
Pacific Islander      3
Name: Race, dtype: int64

In [19]:
df.fired.unique()

array(['Unclear', 'No', 'Yes'], dtype=object)

In [20]:
df.fired.value_counts()

Unclear    1571
Yes         665
No          119
Name: fired, dtype: int64

In [21]:
df = df.dropna(subset = ['fired'])
df.fired.unique()

array(['Unclear', 'No', 'Yes'], dtype=object)

In [22]:
bins = [0, 25, 35, 45, 55, 65, 107]
group_names = ['<25','25-34','35-44','45-54','55-65','>65']


In [23]:
age_range = pd.cut(df['Age'], bins, labels = group_names)
df['age_range'] = pd.cut(df['Age'], bins, labels = group_names)
age_range

0       55-65
1       55-65
2       45-54
3       55-65
4       55-65
        ...  
2350    35-44
2351    55-65
2352    45-54
2353    25-34
2354    25-34
Name: Age, dtype: category
Categories (6, object): [<25 < 25-34 < 35-44 < 45-54 < 55-65 < >65]

In [24]:
blue_state_list = ['Washington','Oregon','California','Nevada','Colorado','New Mexico','Minnesota','Iowa','Wisconsin','Illinois','Wisconsin','Michigan','Pennsylvania','New York','Virginia','New Jersey','Delaware','Washington DC','Maryland','Maine','New Hampshire','Rhode Island', 'Connecticut','Massachusetts','Vermont']
def blue_or_red(state):
    if state in blue_state_list: 
        return "blue"
    else: 
        return "red"
    
df['red/blue state'] = df.State.apply(blue_or_red)

Grouped the states by Red and Blue (Republican / Democrat) based on http://www.politico.com/magazine/story/2014/05/climate-change-map-red-state-blue-state-hot-state-cold-state-107028, these are 2014 categories because the majority of the data in my set is from 2014.

In [25]:
df

,cause_death,State,Gender,Age,Race,armed,priors,fired,age_range,red/blue state
0,Car Accident,New Jersey,Female,64.0,Asian,No,Unclear,Unclear,55-65,blue
1,Car Accident,California,Male,58.0,Asian,Unclear,Unclear,Unclear,55-65,blue
2,Car Accident,Illinois,Female,54.0,Black,No,No,No,45-54,blue
3,Car Accident,Georgia,Male,62.0,Black,No,Unclear,Unclear,55-65,red
4,Car Accident,Michigan,Female,64.0,Black,Unclear,Unclear,Unclear,55-65,blue
...,...,...,...,...,...,...,...,...,...,...
2350,Taser Gun,Florida,Male,41.0,White,No,Yes,Yes,35-44,red
2351,Taser Gun,Kentucky,Male,56.0,White,No,Unclear,Unclear,55-65,red
2352,Taser Gun,North Carolina,Male,55.0,White,No,Unclear,Unclear,45-54,red
2353,Taser Gun,Kansas,Male,34.0,White,No,Unclear,Unclear,25-34,red


In [26]:
df_income = pd.read_csv(os.path.join('C:\Users\morgane1\Desktop\householdincome.csv'))

In [27]:
df_income

,Unnamed: 0,2014 Q4,2015 Q1,2015 Q2,2015 Q3,2015 Q4,2016 Q1,Average
0,Connecticut .................................,"236,828","237,080","240,603","241,352","244,751","246,600",241202.3
1,Maine .......................................,"55,285","54,804","55,841","56,501","57,264","58,054",56291.5
2,Massachusetts ...............................,"404,213","408,040","413,859","416,694","423,950","428,863",415936.5
3,New Hampshire ...............................,"71,515","71,287","72,690","73,606","75,267","76,098",73410.5
4,Rhode Island ................................,"51,952","52,036","52,869","53,209","53,851","54,431",53058.0
...,...,...,...,...,...,...,...,...
46,California ..................................,"1,974,525","2,011,012","2,056,153","2,077,759","2,099,673","2,119,407",2056421.5
47,Hawaii ......................................,"66,378","67,232","68,103","68,861","69,612","70,377",68427.2
48,Nevada ......................................,"118,075","119,273","121,610","122,875","124,484","125,638",121992.5
49,Oregon ......................................,"167,022","169,604","171,874","175,278","176,892","179,076",173291.0


Source of income analysis is the U.S. Bureau of Economic Analysis: http://www.bea.gov/newsreleases/regional/spi/sqpi_newsrelease.htm

In [28]:
df_income['Average '].mean()

300689.362745098

In [29]:
df_income['Average '].median()

186600.5

In [30]:
df_income = df_income.dropna(subset = ['Average '])


In [31]:
df_income['Average '].quantile([.25, .5, .75])

0.25     70918.85
0.50    186600.50
0.75    388273.45
Name: Average , dtype: float64

In [32]:
df_income['Average '].max()

2056421.5

In [33]:
bins = [0, 75000, 200000, 400000, 2056421]
group_names = ['<75000','75000-200000','200000-400000','>400000']

In [34]:
income_range = pd.cut(df_income['Average '], bins, labels = group_names)
df['income_range'] = pd.cut(df_income['Average '], bins, labels = group_names)
income_range

0     200000-400000
1            <75000
2           >400000
3            <75000
4            <75000
          ...      
46              NaN
47           <75000
48     75000-200000
49     75000-200000
50    200000-400000
Name: Average , dtype: category
Categories (4, object): [<75000 < 75000-200000 < 200000-400000 < >400000]

In [43]:
df['income_range'].value_counts()

<75000           14
75000-200000     13
>400000          12
200000-400000    11
Name: income_range, dtype: int64

In [ ]:
#I want to add these incomes to the dataframe based on the state name...not sure how to do that right now. 

In [37]:
df

,cause_death,State,Gender,Age,Race,...,priors,fired,age_range,red/blue state,income_range
0,Car Accident,New Jersey,Female,64.0,Asian,...,Unclear,Unclear,55-65,blue,200000-400000
1,Car Accident,California,Male,58.0,Asian,...,Unclear,Unclear,55-65,blue,<75000
2,Car Accident,Illinois,Female,54.0,Black,...,No,No,45-54,blue,NaN
3,Car Accident,Georgia,Male,62.0,Black,...,Unclear,Unclear,55-65,red,<75000
4,Car Accident,Michigan,Female,64.0,Black,...,Unclear,Unclear,55-65,blue,<75000
...,...,...,...,...,...,...,...,...,...,...,...
2350,Taser Gun,Florida,Male,41.0,White,...,Yes,Yes,35-44,red,NaN
2351,Taser Gun,Kentucky,Male,56.0,White,...,Unclear,Unclear,55-65,red,NaN
2352,Taser Gun,North Carolina,Male,55.0,White,...,Unclear,Unclear,45-54,red,NaN
2353,Taser Gun,Kansas,Male,34.0,White,...,Unclear,Unclear,25-34,red,NaN


In [19]:
y = df.fired
ys = pd.get_dummies(y, prefix = None)
ys

,No,Unclear,Yes
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
2350,0.0,1.0,0.0
2351,0.0,1.0,0.0
2352,0.0,0.0,1.0
2353,0.0,1.0,0.0


In [20]:
#should I see if the total number of deaths is correlated with time?

In [21]:
#drop unnecessary columns ("Photo", "News Story")

In [ ]:
#Split up the city and state column and delete the duplicate city column. 

In [ ]:
#remove all the lines that aren't shootings - tasers.

In [ ]:
#maybe disregard city entirely because that part of the data set isn't complete. 